In [1]:
# test_imports in this cell, restart kernel whenever edits to modules are made

import sys
import os

# Append the src directory to the path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

#import things
from configs import Config
from preprocessing import HiCQuery
from utils import ValidLoop

def test_imports():
    print("testing config class")
    config = Config()
    print(config)

    print("testing 2d contacts preprocessing classes")
    query = HiCQuery(config)
    print(query)

    print("testing helper function imports")
    out = ValidLoop(config, 10000)
    print(out)
    
    #other prints for imported classes

test_imports()

testing config class
testing 2d contacts preprocessing classes
HiC file loaded
testing helper function imports


In [ ]:
## create config to test gm12878 hubs